<a href="https://colab.research.google.com/github/someDeveloperDH/game/blob/DH/RSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from random import shuffle
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
#데이터 경로
base_dir = "/content/drive/MyDrive/proj_rsp/RPS" #필요시 경로 수정 필수

train_dir = os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"test")



train_paper_dir = os.path.join(train_dir,"paper") #650개
train_rock_dir = os.path.join(train_dir,"rock")
train_scissors_dir = os.path.join(train_dir,"scissors")



test_paper_dir = os.path.join(test_dir,"paper") #90개
test_rock_dir = os.path.join(test_dir,"rock")
test_scissors_dir = os.path.join(test_dir,"scissors")

In [ ]:
#데이터 전처리 : 이미지 증강 (Augmentation), validation 데이터 사용
train_datagen = ImageDataGenerator(
    rescale=1./255,            # 이미지의 픽셀 값을 0-1 범위로 스케일링
    rotation_range=40,         # 이미지를 무작위로 40도 회전
    width_shift_range=0.2,     # 이미지를 수평으로 무작위 20% 이동
    height_shift_range=0.2,    # 이미지를 수직으로 무작위 20% 이동
    shear_range=0.2,           # 무작위 전단 변환
    zoom_range=0.2,            # 무작위 확대/축소
    horizontal_flip=True,      # 이미지를 수평으로 뒤집기
    fill_mode='nearest',       # 빈공간의 픽셀 값을 가까운 것과 유사하게 채우기
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale = 1./255)



In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    subset='training',  #subset 설정안하면 분리 안됨
                                                    batch_size=156, #  나머지 없음
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    target_size=(224,224),
                                                    shuffle=True,
                                                    seed=None,
                                                    interpolation='nearest'
                                                    )
validation_generator = train_datagen.flow_from_directory(train_dir,
                                                         subset='validation',
                                                          batch_size=39,      #  배치 크기가 연산 속도와 관련.
                                                          color_mode='rgb',
                                                          class_mode='categorical',
                                                          target_size=(224,224),
                                                          shuffle=True,
                                                          seed=None,
                                                          interpolation='nearest'
                                                              )

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  batch_size=4,
                                                  color_mode='rgb',
                                                  class_mode='categorical',
                                                  target_size=(224,224))



Found 1560 images belonging to 3 classes.
Found 390 images belonging to 3 classes.
Found 270 images belonging to 3 classes.


In [ ]:
#클래스 설정 : 폴더기준으로 자동으로 클래스 나눈거 확인
train_generator.class_indices


{'paper': 0, 'rock': 1, 'scissors': 2}

In [ ]:
#모델생성 : 이미지특징추출 마지막에서 필터 수가 너무 많으면 과적합이 일어날 수 있음. dense가 깊어지면 과적합이 일어날 수 있음.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 flatten_1 (Flatten)         (None, 346112)           

In [ ]:
#모델 컴파일 : 학습 전 단계
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics= ['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch= 10,  # 1560 / 156 = 10
                              epochs=100,
                               validation_steps= 10, # 390 / 39 = 10
                              verbose=2)

<ipython-input-17-07045a86a92d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/100
10/10 - 293s - loss: 3.0713 - accuracy: 0.3667 - val_loss: 1.1258 - val_accuracy: 0.3333 - 293s/epoch - 29s/step
Epoch 2/100
10/10 - 207s - loss: 1.1175 - accuracy: 0.3378 - val_loss: 1.0894 - val_accuracy: 0.3564 - 207s/epoch - 21s/step
Epoch 3/100
10/10 - 225s - loss: 1.0871 - accuracy: 0.3692 - val_loss: 1.1510 - val_accuracy: 0.3333 - 225s/epoch - 23s/step
Epoch 4/100
10/10 - 218s - loss: 1.0999 - accuracy: 0.3449 - val_loss: 1.0866 - val_accuracy: 0.3718 - 218s/epoch - 22s/step
Epoch 5/100
10/10 - 209s - loss: 1.0816 - accuracy: 0.3679 - val_loss: 1.0810 - val_accuracy: 0.3718 - 209s/epoch - 21s/step
Epoch 6/100
10/10 - 218s - loss: 1.0760 - accuracy: 0.3513 - val_loss: 1.0795 - val_accuracy: 0.3128 - 218s/epoch - 22s/step
Epoch 7/100
10/10 - 213s - loss: 1.0710 - accuracy: 0.3423 - val_loss: 1.0763 - val_accuracy: 0.3821 - 213s/epoch - 21s/step
Epoch 8/100
10/10 - 212s - loss: 1.0674 - accuracy: 0.3647 - val_loss: 1.0700 - val_accuracy: 0.3462 - 212s/epoch - 21s/step
